In [1]:
!pip install -U accelerate -q
!pip install -U transformers -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2023.12.2 which is incompatible.


In [2]:
!pip install -U datasets -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2023.10.0 which is incompatible.


In [3]:
import transformers

In [4]:
import pandas as pd

file_path = 'Downloads/twitter_training.csv'
data = pd.read_csv(file_path, header=None)


data.columns = ['ID', 'game', 'label', 'text']


data.sample(5)

,ID,game,label,text
72502,8820,Nvidia,Neutral,Rumor has has it Jensens black spatulas aren a...
29810,726,ApexLegends,Positive,First we had a code sheet and now we have the ...
64029,7771,MaddenNFL,Negative,@EAMaddenNFL spent $10 000 in game currency fo...
20294,12675,WorldOfCraft,Neutral,Bitch thats a wisp from warcraft 3
56722,11332,TomClancysRainbowSix,Positive,"Or Rocket Big League, Sea Queen of Thieves or ..."


In [5]:
cols_to_drop = ['ID','game']
data = data.drop(cols_to_drop,axis=1)
data.sample(5)

,label,text
41615,Irrelevant,<unk>
50893,Irrelevant,Bruh! A lot of Niggaz hate you because you win...
38745,Positive,Really enjoyed the game
72916,Irrelevant,... Congratulations @geethamhp and Dear @niram...
57798,Negative,@Rainbow6Game Please FIX YOUR GAME! I’m from P...


In [6]:
data = data[~data['label'].isin(['Neutral', 'Irrelevant'])]

data.sample(5)

,label,text
11959,Positive,"Caw is dead, all we know now is NBA 2K."
31017,Positive,Bard the Wandering Caretaker n always a good s...
10245,Positive,This guy is incredible.
55916,Negative,someone else who has a problem with
8023,Positive,Since Reinhardt is one of my favorites on Over...


In [7]:
data['label'] = data['label'].replace({'Positive': 1, 'Negative': 0})

In [8]:
data['text'] = data['text'].str.lower()
data.sample(5)

,label,text
70574,1,playing ghost recon breakpoint with no hud no ...
14959,1,i think i'm going to go broke.
70036,1,( and cyber ) punk's right not dropping dead. ...
40882,1,battlefield royal v soundtracks are always the...
65327,0,at last let players grind down your game you g...


In [9]:
data['text'] = data['text'].astype(str)

In [11]:
import re
re_url = re.compile(r'((www.[^\s]+)|(https?://[^\s]+))')
data['text'] = data['text'].apply(lambda x: re_url.sub('', x))

re_letter = re.compile(r"[^a-z\s']")
data['text'] = data['text'].apply(lambda x: re_letter.sub('', x))
data.sample(5)

,label,text
69733,1,oooooooooooooooooooooooooooooooooooooooooooooo...
15274,0,and rhandlerr what is this crap there's no rea...
43542,0,pubgsupport hiya so when'll u be fixing your f...
68082,0,the world people are still dying from covid y...
56241,0,rainbowgame i just ran a ranked game lagged ou...


In [30]:
data=data[:1000]

In [31]:
data.shape

(1000, 2)

In [32]:
from datasets import Dataset

dataset_ = Dataset.from_pandas(data)
dataset = dataset_.train_test_split(0.1)

In [43]:
model_checkpoint = 'distilbert-base-uncased'
batch_size = 64

In [35]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [36]:
def process(x):
  return tokenizer(x['text'])

train_ds = dataset['train'].map(process)
test_ds = dataset['test'].map(process)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [37]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
args = TrainingArguments(
    f'{model_checkpoint}_sentiment_analysis',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 5,
    weight_decay = 0.1,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

In [45]:
from datasets import load_metric
import numpy as np

metric = load_metric('glue', 'sst2')

def compute_metric(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

C:\Anaconda\Lib\site-packages\datasets\load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [46]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

In [47]:
trainer.evaluate([train_ds[0]])

{'eval_loss': 0.7960720062255859,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.0391,
 'eval_samples_per_second': 25.605,
 'eval_steps_per_second': 25.605}

In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.629303,0.670000
2,No log,0.508981,0.670000
3,No log,0.360946,0.850000
4,No log,0.270565,0.940000
5,No log,0.235074,0.960000


TrainOutput(global_step=75, training_loss=0.398921635945638, metrics={'train_runtime': 806.3883, 'train_samples_per_second': 5.58, 'train_steps_per_second': 0.093, 'total_flos': 83907748033056.0, 'train_loss': 0.398921635945638, 'epoch': 5.0})

In [49]:
trainer.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [50]:
import torch
from transformers import DistilBertTokenizer

def predict_sentiment(text, model):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    tokens = tokenizer(text, return_tensors='pt')
    input_ids = tokens['input_ids']

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_ids = input_ids.to(device)

    model = model.to(device)

    with torch.no_grad():
        outputs = model(input_ids)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    if predicted_class == 1:
        return "POSITIVE SENTENCE"
    else:
        return "NEGATIVE SENTENCE"

In [52]:
text_example = "I love this game"
result = predict_sentiment(text_example, trainer.model)
print(result)

POSITIVE SENTENCE
